In [13]:
#! wget -r -e robots=off -P ../ --no-parent  --no-host-directories --reject="index.html*" --convert-links http://tablebase.lichess.ovh/tables/standard/3-4-5/

In [14]:
import chess
from chess import syzygy
from board.BoardEncoder import BoardEncoder
import numpy as np

In [15]:
tablebase = syzygy.Tablebase()
tablebase.add_directory('../tables/standard/3-4-5')

290

## Probing tablebase for selected positions

We are using Syzygy tablebases. More information here: https://python-chess.readthedocs.io/en/latest/syzygy.html

In [16]:
def print_dtz(fen):
    dtz = tablebase.probe_dtz(chess.Board(fen))
    print(f"Distance to checkmate or a zeroing move: {dtz}")


White to move:

![](https://fen2image.chessvision.ai/8/8/8/8/3k4/8/3K4/3Q4_w_-_-_0_1?color=white)


In [17]:
print_dtz('8/8/8/8/3k4/8/3K4/3Q4 w - - 0 1')

Distance to checkmate or a zeroing move: 13


A positive value for DTZ indicates that the side to move needs 13 moves to win (or zero the 50-move counter), so here white is winning.


White to move:

![](https://fen2image.chessvision.ai/k7/p7/8/K7/8/8/8/8_w_-_-_0_1?color=white)


In [18]:
print_dtz('k7/p7/8/K7/8/8/8/8 w - - 0 1')

Distance to checkmate or a zeroing move: 0


Zero means that the position is drawn.

White to move:

![](https://fen2image.chessvision.ai/8/R7/3r4/8/4K3/8/4p3/4k3%20w%20-%20-%200%201?color=white)

In [41]:
print_dtz('8/R7/3r4/8/4K3/8/4p3/4k3 w - - 0 1')

Distance to checkmate or a zeroing move: -10


A negative value indicates that the side to play will lose in 10 moves, assuming the best play by both sides.

## Building a training dataset

In [26]:
def bitarray_to_ndarray(bitarray):
    return np.array([int(bit) for bit in bitarray], dtype=bool)

def put_piece(board, color, piece, check_validity=True):
    while True:
        square = np.random.randint(0, 64)
        while board.piece_at(square) is not None:
            square = np.random.randint(0, 64)
        board.set_piece_at(square, chess.Piece(piece, color)) 
        if board.is_valid() or not check_validity:
            break
        else:
            board.remove_piece_at(square)

def generate_random_fen():
    # Generate a random position, where
    # for each side there is a king and optionally a queen and/or a rook
    
    board = chess.Board()
    board.clear_board()
    board.castling_rights = 0

    # Put kings first because positions without them are illegal
    put_piece(board, chess.WHITE, chess.KING, check_validity=False)
    put_piece(board, chess.BLACK, chess.KING, check_validity=True)

    # Put remaining pieces
    while True:
        color_piece_count = {
            chess.WHITE: {
                chess.QUEEN: np.random.randint(0, 2),
                chess.ROOK: np.random.randint(0, 2),
            },
            chess.BLACK: {
                chess.QUEEN: np.random.randint(0, 2),
                chess.ROOK: np.random.randint(0, 2),
            }
        }
        if sum(color_piece_count[chess.WHITE].values()) + sum(color_piece_count[chess.BLACK].values()) <= 3: # We can have up to 5 pieces, kings included
            break

    for color in chess.COLORS:
        for piece, count in color_piece_count[color].items():
            for _ in range(count):
                put_piece(board, color, piece)
    
    return board.fen()
                

N_SAMPLES = 1000000

X_rows = []
dtzs = []
fens = []

for _ in range(N_SAMPLES):
    fen = generate_random_fen()
    fens.append(fen)
    
    board = chess.Board(fen)
    
    X_row = bitarray_to_ndarray(BoardEncoder.encode(board))
    X_rows.append(X_row)

    dtz = tablebase.probe_dtz(board)
    
    dtzs.append(dtz)

X = np.array(X_rows)
y = np.array(dtzs)  

In [27]:
X.shape

(10000, 837)

In [28]:
y.shape

(10000,)

## Crossvalidation to see if MLP regressor is a good choice

In [30]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score


reg = MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000, verbose=True, alpha=0.5)

cv_score = cross_val_score(reg, X, y, cv=5, scoring='neg_mean_absolute_error')

print(f"Crossvalidation scores: {cv_score}")
print(f"Mean: {cv_score.mean()}")
print(f"Std: {cv_score.std()}")

Iteration 1, loss = 96.59300947
Iteration 2, loss = 90.22306371
Iteration 3, loss = 72.03628915
Iteration 4, loss = 65.28098666
Iteration 5, loss = 61.46189199
Iteration 6, loss = 57.72574740
Iteration 7, loss = 54.00354664
Iteration 8, loss = 48.83017401
Iteration 9, loss = 42.72554712
Iteration 10, loss = 35.15343415
Iteration 11, loss = 26.96611058
Iteration 12, loss = 19.95710039
Iteration 13, loss = 15.37449698
Iteration 14, loss = 11.52068889
Iteration 15, loss = 9.42722572
Iteration 16, loss = 7.70166278
Iteration 17, loss = 6.04589562
Iteration 18, loss = 5.00544593
Iteration 19, loss = 4.28561283
Iteration 20, loss = 3.58894316
Iteration 21, loss = 3.13664374
Iteration 22, loss = 3.00984908
Iteration 23, loss = 2.64135122
Iteration 24, loss = 2.24453254
Iteration 25, loss = 1.94692823
Iteration 26, loss = 1.72638762
Iteration 27, loss = 1.64737019
Iteration 28, loss = 1.53859224
Iteration 29, loss = 1.42076567
Iteration 30, loss = 1.35511971
Iteration 31, loss = 1.28496456
Ite

## Proper training

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
reg.fit(X_train, y_train)
mse = reg.score(X_test, y_test)

print(f"Mean squared error: {mse}")

Iteration 1, loss = 94.58007331
Iteration 2, loss = 85.65828131
Iteration 3, loss = 68.16725190
Iteration 4, loss = 62.65989173
Iteration 5, loss = 58.84965088
Iteration 6, loss = 54.91891233
Iteration 7, loss = 49.94607768
Iteration 8, loss = 44.11239434
Iteration 9, loss = 38.24427050
Iteration 10, loss = 31.60053273
Iteration 11, loss = 24.69689862
Iteration 12, loss = 18.35706284
Iteration 13, loss = 13.76086822
Iteration 14, loss = 10.39815565
Iteration 15, loss = 8.11397679
Iteration 16, loss = 6.51892050
Iteration 17, loss = 5.12757897
Iteration 18, loss = 4.51856799
Iteration 19, loss = 3.80975624
Iteration 20, loss = 3.30658826
Iteration 21, loss = 2.93709566
Iteration 22, loss = 2.68098260
Iteration 23, loss = 2.38312321
Iteration 24, loss = 2.17916290
Iteration 25, loss = 1.98858801
Iteration 26, loss = 1.86731349
Iteration 27, loss = 1.81454412
Iteration 28, loss = 1.76382826
Iteration 29, loss = 1.75342055
Iteration 30, loss = 1.67885496
Iteration 31, loss = 1.64852994
Ite

## Model predictions

In [35]:
def get_estimated_dtz(reg, fen):
    board = chess.Board(fen)
    X = bitarray_to_ndarray(BoardEncoder.encode(board)).reshape(1, -1)
    return reg.predict(X)[0]


White to move:

![](https://fen2image.chessvision.ai/8/8/8/8/3k4/8/3K4/3Q4_w_-_-_0_1?color=white)

In [36]:
get_estimated_dtz(reg, '8/8/8/8/3k4/8/3K4/3Q4 w - - 0 1')

23.265909446173875

Here, the model correctly estimates that white is winning, however in fact white only needs 13 moves to win while the model estimates 23.


White to move:

![](https://fen2image.chessvision.ai/k7/p7/8/K7/8/8/8/8_w_-_-_0_1?color=white)

In [38]:
get_estimated_dtz(reg, 'k7/p7/8/K7/8/8/8/8 w - - 0 1')

-5.527810798758446

The model estimates black winning in 5 moves, whereas in fact the position is drawn.

White to move:

![](https://fen2image.chessvision.ai/8/R7/3r4/8/4K3/8/4p3/4k3%20w%20-%20-%200%201?color=white)

In [40]:
get_estimated_dtz(reg, '8/R7/3r4/8/4K3/8/4p3/4k3 w - - 0 1')

-9.935931977421188

Here the model is correct - in fact, black needs 10 moves to win.